In [1]:
from librosa import load 
from librosa import feature
import soundfile
import os, glob, pickle
import numpy as np
import resampy
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [2]:
def extract_feature(file_name, mfcc=True, chroma=True, mel=True):
    # Assuming X is the audio signal loaded from the file
    X, sample_rate = load(file_name, res_type='kaiser_fast')

    result = np.array([])

    if mfcc:
        mfccs = np.mean(feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
        result = np.hstack((result, mfccs))

    if chroma:
        chroma = np.mean(feature.chroma_stft(y=X, sr=sample_rate).T, axis=0)
        result = np.hstack((result, chroma))

    if mel:
        mel = np.mean(feature.melspectrogram(y=X, sr=sample_rate).T, axis=0)
        result = np.hstack((result, mel))

    return result


In [3]:
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}
#Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [4]:
# Assuming emotions and observed_emotions are defined somewhere in your code

# Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x, y = [], []

    # Update the directory path to match your dataset
    for file in glob.glob("/home/anshu/speech-emotion-recognition-ravdess-data/Actor_*/*.wav"):
        file_name = os.path.basename(file)
        emotion = emotions[file_name.split("-")[2]]

        if emotion not in observed_emotions:
            continue

        # Assuming extract_feature is correctly implemented
        feature = extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)

    # Split the data into training and testing sets
    print("File Names:", [os.path.basename(file) for file in glob.glob("/home/anshu/speech-emotion-recognition-ravdess-data/Actor_*/*.wav")])
    print("Number of samples before split:", len(x))
    return train_test_split(x, y, test_size=test_size, random_state=9)


In [5]:
#Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

File Names: ['03-01-04-02-01-01-02.wav', '03-01-08-02-01-01-02.wav', '03-01-04-01-01-02-02.wav', '03-01-05-01-01-02-02.wav', '03-01-07-02-02-01-02.wav', '03-01-03-01-01-01-02.wav', '03-01-04-02-02-02-02.wav', '03-01-06-02-01-01-02.wav', '03-01-05-01-02-01-02.wav', '03-01-03-02-01-02-02.wav', '03-01-07-01-01-02-02.wav', '03-01-04-01-02-01-02.wav', '03-01-05-02-01-02-02.wav', '03-01-02-02-02-02-02.wav', '03-01-07-01-02-02-02.wav', '03-01-03-01-02-01-02.wav', '03-01-06-01-01-02-02.wav', '03-01-03-01-02-02-02.wav', '03-01-05-01-01-01-02.wav', '03-01-05-02-02-01-02.wav', '03-01-03-01-01-02-02.wav', '03-01-05-01-02-02-02.wav', '03-01-08-02-02-01-02.wav', '03-01-07-02-01-02-02.wav', '03-01-07-02-02-02-02.wav', '03-01-02-02-01-02-02.wav', '03-01-05-02-02-02-02.wav', '03-01-06-01-02-01-02.wav', '03-01-03-02-02-01-02.wav', '03-01-08-01-01-02-02.wav', '03-01-06-02-02-02-02.wav', '03-01-01-01-02-01-02.wav', '03-01-03-02-02-02-02.wav', '03-01-02-01-02-01-02.wav', '03-01-02-02-02-01-02.wav', '03-01-

In [6]:
#Get the shape of the training and testing datasets
print((x_train[0], x_test[0]))

(array([-5.65453979e+02,  4.51302719e+01, -2.86863575e+01,  1.35358400e+01,
       -1.87467937e+01,  7.32392550e-01, -1.12955465e+01, -1.29123163e+01,
       -1.10600948e+01, -6.55311871e+00, -3.62638307e+00, -1.21256104e+01,
       -2.66078830e+00, -4.27143574e+00, -3.60851026e+00, -6.09255838e+00,
       -1.10117102e+00, -1.72855103e+00,  1.38818300e+00,  8.56476498e+00,
        8.03161335e+00,  1.09084063e+01,  3.94344473e+00,  2.36860681e+00,
        1.11145866e+00,  1.63311815e+00,  6.73949778e-01, -1.39186418e+00,
       -1.93602026e+00, -3.67748404e+00, -2.27698416e-01, -1.53320312e+00,
       -2.20388913e+00, -4.20142078e+00, -1.94263422e+00,  5.55684566e-01,
       -1.53636658e+00, -9.88395393e-01, -2.48997450e-01,  2.89547801e+00,
        3.06930363e-01,  2.95509338e-01,  3.34442228e-01,  3.49008769e-01,
        4.45362389e-01,  4.61242825e-01,  3.23804468e-01,  3.20300609e-01,
        2.91187793e-01,  2.76768863e-01,  2.85780668e-01,  2.81729639e-01,
        5.72338013e-06, 

In [ ]:
#Get the number of features extracted
print(f'Features extracted: {x_train[1]}')

In [7]:
#Train the model
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [8]:
#Predict for the test set
y_pred=model.predict(x_test)

In [9]:
#Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#Print the accuracy

accuracy = int(accuracy*100)

print(f"Accuracy:{accuracy}" )



Accuracy:72


In [10]:

if accuracy > 55:
    
    print("Observed emotions are : ",observed_emotions)
    
else:
    
    print("Observed emotions are not observed by the classifier ")
    

Observed emotions are :  ['calm', 'happy', 'fearful', 'disgust']
